In [1]:
import pyLDAvis

TRAIN_LDA = False
lda_model_path = '../data/2016-10-20-news-lda100.pkl'
pyldavis_html_path = './2016-10-20-news-lda100-pyldavis.html'

/home/lovit/anaconda2/envs/scrapper/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
import pickle

data_path = '../data/2016-10-20-news-bow.pkl'

with open(data_path, 'rb') as f:
    params = pickle.load(f)
    x = params['x']
    index2word = params['index2word']
    word2index = params['word2index']

print(x.shape)

(30091, 9774)


In [3]:
import warnings

# LDA warning off
warnings.filterwarnings('ignore')

In [4]:
import gensim
from gensim.models import LdaModel
from gensim.corpora.dictionary import Dictionary

corpus = gensim.matutils.Sparse2Corpus(x, documents_columns=False)
dictionary = Dictionary.from_corpus(
    corpus,
    id2word = dict(enumerate(index2word))
)

print('corpus length = {}'.format(len(corpus)))

corpus length = 30091


In [12]:
# from text to corpus (1 / 2)
corpus_path = '2016-10-20_article_all_normed_nountokenized.txt'
min_count = 40

class Documents:
    def __init__(self, path):
        self.path = path
    def __iter__(self):
        with open(self.path, encoding='utf-8') as f:
            for doc in f:
                yield doc.strip().split()

import gensim
documents = Documents(corpus_path)
dictionary = gensim.corpora.Dictionary(documents)
print('dictionary size : %d' % len(dictionary))

from collections import Counter
word_counter = Counter((word for words in documents for word in words))
removal_word_idxs = {
    dictionary.token2id[word] for word, count in word_counter.items()
    if count < min_count
}
dictionary.filter_tokens(removal_word_idxs)
dictionary.compactify()
print('dictionary size : %d' % len(dictionary))

dictionary size : 37987
dictionary size : 10354


In [17]:
# from text to corpus (2 / 2)
class Corpus:
    def __init__(self, path, dictionary):
        self.path = path
        self.dictionary = dictionary
        self.length = 0
    def __iter__(self):
        with open(self.path, encoding='utf-8') as f:
            for doc in f:
                yield self.dictionary.doc2bow(doc.split())
    def __len__(self):
        if self.length == 0:
            with open(self.path, encoding='utf-8') as f:
                for i, doc in enumerate(f):
                    continue
            self.length = i + 1
        return self.length

corpus = Corpus(corpus_path, dictionary)
for i, doc in enumerate(corpus):
    if i >= 5: break
    print(doc[:10])

[]
[(0, 1), (1, 1), (2, 1), (3, 1)]
[(4, 1), (5, 4), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 2), (12, 1), (13, 1)]
[(5, 1), (53, 1), (91, 1), (93, 2), (136, 2), (147, 4), (148, 1), (149, 1), (150, 4), (151, 1)]
[(5, 1), (93, 2), (149, 1), (182, 1), (187, 1), (188, 1), (189, 1), (190, 1), (191, 1), (192, 1)]


In [5]:
if TRAIN_LDA:
    lda_model = LdaModel(corpus, id2word=dictionary, num_topics=100)
    with open(lda_model_path, 'wb') as f:
        pickle.dump(lda_model, f)
else:
    with open(lda_model_path, 'rb') as f:
        lda_model = pickle.load(f)

In [6]:
lda_model.print_topic(0)

'0.103*"보는" + 0.038*"세계" + 0.029*"변신" + 0.028*"시민들" + 0.026*"저녁" + 0.020*"축제" + 0.020*"온라인" + 0.014*"199" + 0.013*"거리" + 0.013*"우정"'

In [7]:
print('topic prob P(t), shape = {}'.format(lda_model.alpha.shape))
print('topic prob P(t), sum = {}'.format(lda_model.alpha.sum()))

topic prob P(t), shape = (100,)
topic prob P(t), sum = 0.9999998211860657


In [8]:
# for doc-topic vector, use inference function
# gamma = topic_model.inference(corpus)

In [9]:
print('topic-term matrix')
print(lda_model.expElogbeta.shape)

topic-term matrix
(100, 9774)


In [6]:
import pyLDAvis
import pyLDAvis.gensim as gensimvis

prepared_data = gensimvis.prepare(lda_model, corpus, dictionary)

In [7]:
type(prepared_data)

pyLDAvis._prepare.PreparedData

In [8]:
pyLDAvis.display(prepared_data)

In [9]:
pyLDAvis.save_html(prepared_data, pyldavis_html_path)